In [1]:
##Import packages:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import sklearn as sk
import seaborn as sns
import tensorflow as tf
import keras as K

/home/s2c/anaconda3/envs/ASHRAE/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
##Import data:
weather_train=pd.read_csv('data/weather_train.csv')
weather_test=pd.read_csv('data/weather_test.csv')
# test=pd.read_csv('data/test.csv')
train=pd.read_csv('data/train.csv')
sample_submission=pd.read_csv('data/sample_submission.csv')
building_metadata=pd.read_csv('data/building_metadata.csv')
print("Import done")

Import done


In [3]:
##Merge into one dataframe:
df=train.merge(building_metadata)
df=df.merge(weather_train)
df['timestamp'] = pd.to_datetime(df['timestamp'])
print("Merge done")

Merge done


In [4]:
##New features:
#Time:
df['hour']=df.timestamp.dt.hour
df['month']=df.timestamp.dt.month
df['year']=df.timestamp.dt.year
df['day']=df.timestamp.dt.weekday
df['weekend'] = np.where((df['day'] == 5) | (df['day'] == 6), 1, 0)
print("Feature engineering done")

Feature engineering done


In [5]:
##One-hot encoding:
#Meter:
df = pd.get_dummies(df, columns=['meter'])
#Primary-use:
categorical_feature_mask = df.dtypes==object
categorical_cols = df.columns[categorical_feature_mask].tolist()
le = LabelEncoder()
df[categorical_cols] = df[categorical_cols].apply(lambda col: le.fit_transform(col))
df = pd.get_dummies(df, columns=['primary_use'])
print("One-hot encoding done")

One-hot encoding done


In [6]:
##Missing values:
print('Missing values for the dataset (in %)\n ')
print(df.isnull().sum())

Missing values for the dataset (in %)
 
building_id                  0
timestamp                    0
meter_reading                0
site_id                      0
square_feet                  0
year_built            12113306
floor_count           16630052
air_temperature           6163
cloud_coverage         8734870
dew_temperature           9645
precip_depth_1_hr      3658528
sea_level_pressure     1141174
wind_direction         1358553
wind_speed               53181
hour                         0
month                        0
year                         0
day                          0
weekend                      0
meter_0                      0
meter_1                      0
meter_2                      0
meter_3                      0
primary_use_0                0
primary_use_1                0
primary_use_2                0
primary_use_3                0
primary_use_4                0
primary_use_5                0
primary_use_6                0
primary_use_7                0

In [7]:
##Handling missing data:

#Removing features with missing values for more than 30% of the dataset
dropped_features=["cloud_coverage","precip_depth_1_hr","year_built","floor_count"]
for x in dropped_features:
    df=df.drop(columns=x)
    
#Replacing missing values with median:
replace_median=["sea_level_pressure","wind_direction","air_temperature","dew_temperature","wind_speed"]
for x in replace_median:
    df[x].fillna(df[x].median(), inplace=True)
    
print("Missing values handled")

Missing values handled


In [8]:
##Dropping irrelevant features:
irrelevant=["timestamp","year"]
for x in irrelevant:
    df=df.drop(columns=x)

In [9]:
##Standarizing
real_values=["square_feet","wind_direction","wind_speed","air_temperature","sea_level_pressure","dew_temperature"]
for x in real_values:
    df[x]=(df[x]-df[x].min())/(df[x].max()-df[x].min())
df.head()
print("Standarizing done")

Standarizing done


In [10]:
##Reducing memory usage:

#Float
float_mask = df.dtypes==float
float_cols = df.columns[float_mask].tolist()
df[float_cols]=df[float_cols].apply(pd.to_numeric,downcast='float')

#Int
int_mask = df.dtypes==int
int_cols = df.columns[int_mask].tolist()
df[int_cols]=df[int_cols].apply(pd.to_numeric,downcast='unsigned')

print("Memory usage reduced")

Memory usage reduced


In [11]:
df.head()

,building_id,meter_reading,site_id,square_feet,air_temperature,dew_temperature,sea_level_pressure,wind_direction,wind_speed,hour,...,primary_use_6,primary_use_7,primary_use_8,primary_use_9,primary_use_10,primary_use_11,primary_use_12,primary_use_13,primary_use_14,primary_use_15
0,0,0.0,0,0.008173,0.708279,0.900164,0.666235,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0.0,0,0.002786,0.708279,0.900164,0.666235,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0.0,0,0.005822,0.708279,0.900164,0.666235,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0.0,0,0.026754,0.708279,0.900164,0.666235,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0.0,0,0.132985,0.708279,0.900164,0.666235,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
##Transforming targeted value
df.meter_reading=np.log1p(df.meter_reading)

In [13]:
#Distribution of the log of targeted value:
df.meter_reading.hist(bins=50)

In [14]:
## Correlation matrix plot:
corr=df.corr()
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)
#Degeulasse! 

In [15]:
#Defining training data:
no_feature = ['building_id', 'timestamp', 'meter_reading', 'site_id']
X_train = df[df.columns.difference(no_feature)].copy(deep=False)
y_train = df['meter_reading']

In [17]:
#Dump xTrain and yTrain to local file
X_train.to_pickle('xTrain.data')
y_train.to_pick('yTrain.data')

AttributeError: 'Series' object has no attribute 'to_pick'